In [1]:
import subprocess
import time
import random
import pandas as pd

## Evaluating One Candidate

In [2]:
def apply_passes(input, output, pass_order):
    # passes_str = " ".join(pass_order)
    cmd = ["riscv64-unknown-elf-gcc", "-I./support/", "-O2"]
    cmd+= pass_order
    cmd+= [input, "./support/beebsc.c", "./support/main.c", "-lm", "-march=rv64imaf", "-mabi=lp64", "-o", output]
    # print(" ".join(cmd))
    subprocess.run(cmd, capture_output=True, text=True, check=True)

def measure_runtime(binary, count=100):
    total_time = 0.0
    for _ in range(count):
        start = time.time()
        result = subprocess.run([f"./{binary}"], capture_output=True, text=True, check=True)
        if result.stdout.strip() == "Benchmark completed successfully with no errors.":
            pass
        else:
            raise ValueError("Benchmark did not complete successfully.")
        end = time.time()
        total_time += (end - start)
    return total_time / count

def measure_size(binary):
    result = subprocess.run(["size", binary], capture_output=True, text=True)
    lines = result.stdout.strip().split("\n")
    if len(lines) < 2:
        return None
    size_fields = lines[1].split()
    total_size = sum(int(x) for x in size_fields[:4])
    return total_size

def evaluate_candidate(pass_order, input, workdir, mode="runtime"):
    candidate = f"{workdir}/candidate_bin"
    try:
        apply_passes(input, candidate, pass_order)
        if mode == "runtime":
            fitness = measure_runtime(candidate)
        elif mode == "size":
            fitness = measure_size(candidate)
        else:
            raise ValueError("Invalid mode")
        return fitness
    except subprocess.CalledProcessError:
        return float("inf")  # Penalize failure

## Genetic Algorithm Operators

In [3]:

def random_candidate(pass_pool, max_len=5):
    length = random.randint(1, max_len)
    return random.sample(pass_pool, length)



def crossover(parent1, parent2, max_len=5):
    # Pick cut points
    p1_cut = random.randint(1, len(parent1))
    p2_cut = random.randint(1, len(parent2))

    # Combine slices
    child = parent1[:p1_cut] + parent2[p2_cut:]

    # Remove duplicates while preserving order
    seen = set()
    deduped_child = []
    for p in child:
        if p not in seen:
            deduped_child.append(p)
            seen.add(p)

    # Clip to max length
    return deduped_child[:max_len]


def mutate(candidate, pass_pool, mutation_rate=0.1, max_len=5):
    # Start with a copy
    mutated = candidate[:]
    
    # Replace existing passes
    for i in range(len(mutated)):
        if random.random() < mutation_rate:
            # Choose a new pass not in the candidate
            available = list(set(pass_pool) - set(mutated))
            if available:
                mutated[i] = random.choice(available)

    # Random insertion
    if len(mutated) < max_len and random.random() < mutation_rate:
        available = list(set(pass_pool) - set(mutated))
        if available:
            insert_pos = random.randint(0, len(mutated))
            mutated.insert(insert_pos, random.choice(available))

    # Random deletion
    if len(mutated) > 1 and random.random() < mutation_rate:
        del_pos = random.randint(0, len(mutated) - 1)
        mutated.pop(del_pos)

    return mutated



## Running the GA loop

In [4]:
import pandas as pd

def run_ga(input_bc, pass_pool, generations=10, pop_size=20, seq_length=5):
    population = [random_candidate(pass_pool, seq_length) for _ in range(pop_size)]
    stats = []

    for generation in range(generations):
        print(f"Generation {generation}")
        fitnesses = []
        for candidate in population:
            fitness = evaluate_candidate(candidate, input_bc, "./scratch")
            fitnesses.append((fitness, candidate))
            print(f"Candidate {candidate} => Fitness {fitness}")

        # Selection
        fitnesses.sort(key=lambda x: x[0])
        population = [cand for _, cand in fitnesses[:(pop_size // 2 - 5)]]
        # Add the worst 5 candidates to the population
        population += [cand for _, cand in fitnesses[-5:]]

        # Crossover and mutation
        new_population = []
        while len(new_population) < pop_size:
            parents = random.sample(population, 2)
            c1 = crossover(parents[0], parents[1])
            c1 = mutate(c1, pass_pool)
            new_population.append(c1)

        population = new_population

        # Collect stats for the generation
        best_fitness, best_candidate = fitnesses[0]
        worst_fitness, worst_candidate = fitnesses[-1]
        avg_fitness = sum(f[0] for f in fitnesses) / len(fitnesses)
        stats.append({
            "Generation": generation,
            "Best Candidate": best_candidate,
            "Worst Candidate": worst_candidate,
            "Best Time": best_fitness,
            "Worst Time": worst_fitness,
            "Average Time": avg_fitness
        })

    # Final best
    best = min(fitnesses, key=lambda x: x[0])
    print(f"Best candidate: {best[1]} with fitness {best[0]}")

    # Create a DataFrame from stats
    stats_df = pd.DataFrame(stats)
    # print(stats_df)
    return stats_df


## Example usage

In [5]:
PASS_POOL = [
    "-fgcse-after-reload",
    "-fipa-cp-clone",
    "-floop-interchange",
    "-floop-unroll-and-jam",
    "-fpeel-loops",
    "-fpredictive-commoning",
    "-fsplit-loops",
    "-fsplit-paths",
    "-ftree-loop-distribution",
    "-ftree-partial-pre",
    "-funswitch-loops",
    "-fvect-cost-model=dynamic",
    "-fversion-loops-for-strides",
]

In [6]:
sources = ["src/aha-mont64/mont64.c"]


stats_df = run_ga(sources[0], PASS_POOL, generations=30, pop_size=100, seq_length=5)

Generation 0
Candidate ['-fipa-cp-clone'] => Fitness 0.005118801593780518
Candidate ['-floop-interchange', '-fpredictive-commoning', '-fsplit-paths'] => Fitness 0.004726970195770263
Candidate ['-fpredictive-commoning', '-fversion-loops-for-strides'] => Fitness 0.004761526584625244
Candidate ['-fsplit-paths', '-fsplit-loops', '-fipa-cp-clone', '-fpeel-loops', '-floop-interchange'] => Fitness 0.004699499607086182
Candidate ['-floop-unroll-and-jam', '-ftree-partial-pre'] => Fitness 0.005052263736724853
Candidate ['-fvect-cost-model=dynamic', '-fgcse-after-reload'] => Fitness 0.004902186393737793
Candidate ['-ftree-loop-distribution', '-fvect-cost-model=dynamic'] => Fitness 0.004821047782897949
Candidate ['-ftree-loop-distribution', '-fsplit-loops'] => Fitness 0.005024423599243164
Candidate ['-fsplit-loops'] => Fitness 0.004759070873260498
Candidate ['-fgcse-after-reload'] => Fitness 0.004909267425537109
Candidate ['-ftree-loop-distribution', '-funswitch-loops', '-fsplit-paths'] => Fitness

In [7]:
stats_df

,Generation,Best Candidate,Worst Candidate,Best Time,Worst Time,Average Time
0,0,"[-floop-interchange, -fpredictive-commoning, -...","[-fgcse-after-reload, -fvect-cost-model=dynami...",0.004627,0.005139,0.004719
1,1,"[-fversion-loops-for-strides, -fpeel-loops, -f...",[-ftree-loop-distribution],0.004609,0.004924,0.004677
2,2,"[-floop-interchange, -funswitch-loops, -floop-...",[-fgcse-after-reload],0.004612,0.005336,0.004684
3,3,"[-fsplit-loops, -floop-interchange]",[-fipa-cp-clone],0.004603,0.004849,0.004676
4,4,[-floop-unroll-and-jam],"[-fgcse-after-reload, -fipa-cp-clone]",0.004610,0.005038,0.004678
5,5,"[-fversion-loops-for-strides, -floop-interchange]","[-fipa-cp-clone, -funswitch-loops, -fsplit-paths]",0.004613,0.004912,0.004680
6,6,[-fgcse-after-reload],[-fversion-loops-for-strides],0.004615,0.004965,0.004679
7,7,[-fpredictive-commoning],[-fpeel-loops],0.004626,0.005321,0.004685
8,8,"[-floop-interchange, -fpeel-loops]","[-fsplit-paths, -fipa-cp-clone, -fpeel-loops]",0.004598,0.005101,0.004674
9,9,"[-fpredictive-commoning, -fversion-loops-for-s...",[-fpredictive-commoning],0.004610,0.004926,0.004667


In [47]:
stats_df

,Generation,Best Candidate,Worst Candidate,Best Time,Worst Time,Average Time
0,0,"[-fvect-cost-model=dynamic, -funswitch-loops, ...",[-fipa-cp-clone],0.004478,0.005760,0.004828
1,1,"[-ftree-partial-pre, -fpredictive-commoning, -...","[-fsplit-paths, -fpredictive-commoning]",0.004466,0.005619,0.004603
2,2,"[-fversion-loops-for-strides, -ftree-loop-dist...",[-fipa-cp-clone],0.004494,0.005263,0.004587
3,3,"[-floop-unroll-and-jam, -floop-interchange, -f...","[-fversion-loops-for-strides, -ftree-loop-dist...",0.004505,0.005466,0.004591
4,4,[-fsplit-paths],"[-fvect-cost-model=dynamic, -ftree-loop-distri...",0.004495,0.005355,0.004599
5,5,"[-fversion-loops-for-strides, -ftree-loop-dist...",[-fvect-cost-model=dynamic],0.004482,0.005418,0.004595
6,6,[-fvect-cost-model=dynamic],"[-funswitch-loops, -ftree-partial-pre]",0.004500,0.005673,0.004665
7,7,[-fvect-cost-model=dynamic],"[-floop-unroll-and-jam, -ftree-loop-distributi...",0.004512,0.005101,0.004666
8,8,"[-fvect-cost-model=dynamic, -fipa-cp-clone, -f...","[-floop-unroll-and-jam, -fsplit-paths, -ftree-...",0.004518,0.005535,0.004724
9,9,"[-fvect-cost-model=dynamic, -fsplit-loops, -fp...",[-floop-interchange],0.004549,0.005435,0.004682


In [49]:
stats_df.iloc[2]

Generation                                                         2
Best Candidate     [-fversion-loops-for-strides, -ftree-loop-dist...
Worst Candidate                                     [-fipa-cp-clone]
Best Time                                                   0.004494
Worst Time                                                  0.005263
Average Time                                                0.004587
Name: 2, dtype: object